<a href="https://colab.research.google.com/github/Leoby01/LG-AImers/blob/main/LGAImers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
seed_everything(37) #Seed 고정

In [4]:
filename = '/content/drive/My Drive/LGAImers/train.csv'
train_csv = pd.read_csv(filename)
filename = '/content/drive/My Drive/LGAImers/test.csv'
test_csv = pd.read_csv(filename)
#train_csv
#  Product_ID : 제품의 고유 ID
#  Y_Class : 제품의 품질상태(Target)
#    0 : 적정 기준 미달 (부적합)
#    1 : 적합
#    2 : 적정 기준 초과 (부적합)
#  Y_Quality : 제품 품질 관련 정량적 수치
#  TIMESTAMP : 제품이 공정에 들어간 시각
#  LINE : 제품이 들어간 공정 LINE의 종류('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)
#  PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)
#test_csv
#  PRODUCT_ID : 제품의 고유 ID
#  TIMESTAMP : 제품이 공정에 들어간 시각
#  LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재)
#  PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재)
#  X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수
#sample_submission.csv[제출 양식]
#  PRODUCT_ID : 제품의 고유 ID
#  Y_Class : 예측한 제품 품질 상태
#    0 : 적정 기준 미달 (부적합)
#    1 : 적합
#    2 : 적정 기준 초과 (부적합)

In [7]:
train_x = train_csv.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_csv['Y_Class']
test_x = test_csv.drop(columns = ['PRODUCT_ID', 'TIMESTAMP'])

In [8]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [10]:
#qualitive to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
  le = LabelEncoder()
  le = le.fit(train_x[i])
  train_x[i] = le.transform(train_x[i])

  for label in np.unique(test_x[i]):
    if label not in le.classes_:
      le.classes_ = np.append(le.classes_, label)
  test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [12]:
RF = RandomForestClassifier(random_state = 37).fit(train_x, train_y)
print('Done.')

Done.


In [13]:
preds = RF.predict(test_x)
print('Done.')

Done.


In [17]:
submit = pd.read_csv('/content/drive/My Drive/LGAImers/sample_submission.csv')

In [18]:
submit['Y_Class'] = preds

In [19]:
submit.to_csv('/content/drive/My Drive/LGAImers/baseline_submission.csv', index = False)